# Assignment 1

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: POS tagging, Sequence labelling, RNNs

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import shutil
import urllib
import zipfile
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.metrics import (
    f1_score,
    classification_report,
)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Embedding, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

2024-03-09 15:40:38.874487: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-09 15:40:38.925404: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-09 15:40:38.926148: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 15:40:39.860633: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def set_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # os.environ["TF_DETERMINISTIC_OPS"] = "1" ### can make training slower


set_reproducibility(seed=42)

# [Task 1 - 0.5 points] Corpus

### Download the corpus

In [3]:
def download_url(download_path: Path, url: str):
    urllib.request.urlretrieve(url, filename=download_path)

In [4]:
dataset_url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"
dataset_name = "dependency_treebank"

dataset_folder = Path.cwd().joinpath("Datasets")
if not dataset_folder.exists():
    dataset_folder.mkdir(parents=True)

dataset_zip_path = dataset_folder.joinpath("dependency_treebank.zip")
if not dataset_zip_path.exists():
    print("Downloading dataset... ", end="")
    download_url(url=dataset_url, download_path=dataset_zip_path)
    print("Download complete!")
else:
    print("Dataset already downloaded!")
dataset_path = dataset_folder.joinpath(dataset_name)

if not dataset_path.exists():
    print("Extracting dataset... (it may take a while...) ", end="")
    shutil.unpack_archive(dataset_zip_path, dataset_folder)
    print("Extraction completed!")
else:
    print("Dataset already extracted!")

Dataset already downloaded!
Dataset already extracted!


#### Encode the corpus into a pandas DataFrame object

In [6]:
folder = dataset_folder.joinpath(dataset_name)


dataframe_rows = []
for file_path in sorted(folder.glob("*.dp")):
    with file_path.open(mode="r", encoding="utf-8") as text_file:
        text = text_file.read()
        ### Split sentences (\n\n is used for most NLP datasets to split sentences)
        sentences = text.split("\n\n")

        ### Observing each sentence
        for s in sentences:
            sentence = []
            tags = []
            ### Taking every line
            for line in s.split("\n"):
                columns = line.split("\t")
                ### If every line have word, tag, value
                if len(columns) > 2:
                    ### Put words and tags into lists
                    sentence.append(columns[0])
                    tags.append(columns[1])

            file_id = int(file_path.stem.split("_")[1])
            dataframe_row = {"file_id": file_id, "sentence": sentence, "tag": tags}
            dataframe_rows.append(dataframe_row)

df = pd.DataFrame(dataframe_rows)

FILE_ID, SENTENCE, TAGS = df.columns.values

In [7]:
df.head()

,file_id,sentence,tag
0,1,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ..."
1,1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ..."
2,2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP..."
3,3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS..."
4,3,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V..."


#### Splitting Data Train-Test-Validation

In [8]:
### File indices for train/validation/test
train_ids = np.arange(1, 101)
val_ids = np.arange(101, 151)
test_ids = np.arange(151, 200)

df_train = df[df[FILE_ID].isin(train_ids)]
df_val = df[df[FILE_ID].isin(val_ids)]
df_test = df[df[FILE_ID].isin(test_ids)]

# [Task 2 - 0.5 points] Text encoding

To train a neural POS tagger, you first need to encode text into numerical format.

### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.
* [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

In [9]:
### Download Glove

embedding_dim = 100
glove_file = f"glove.6B.{embedding_dim}d.txt"
glove_zip = "glove.6B.zip"
glove_path = Path.cwd().joinpath(glove_file)
if not glove_path.exists():
    urllib.request.urlretrieve(
        "http://nlp.stanford.edu/data/glove.6B.zip", Path.cwd().joinpath(glove_zip)
    )
    with zipfile.ZipFile(glove_zip, "r") as zip_ref:
        zip_ref.extractall()

In [10]:
embeddings_index = {}
with open(glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [11]:
# max_sequence_length = int(np.quantile([len(seq) for seq in df_train["sentence"]], 0.99))
max_sequence_length = max([len(seq) for seq in df_train[SENTENCE]])

# TODO remove
hparams = {
    "batch_size": 128,
    "embedding_dim": 100,
    "embedding_trainable": False,
    "learning_rate": 0.005,
    "max_sequence_length": max_sequence_length,
    "vocab_size": 7405,
    "tag_size": 46,
}

In [12]:
### Preprocessing: lowercase
def make_lower(df):
    df[SENTENCE].apply(lambda l: [i.lower() for i in l])


make_lower(df_train)
make_lower(df_val)
# TODO va bene fare lowercase sul test set? vediamo se va senza
# make_lower(df_test)

In [13]:
### Use Keras Tokenizer to create Vocabulary

# TODO ci hanno consigliato codice deprecato (tokenizer) io vorrei usare TextVectorization ma fa schifo

tokenizer = Tokenizer(oov_token="OOV")
tokenizer.fit_on_texts(df_train["sentence"])

tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(df_train["tag"])


### Turn text into into padded sequences.
def prep_text(texts, tokenizer, max_sequence_length):
    text_sequences = tokenizer.texts_to_sequences(texts)
    return sequence.pad_sequences(
        text_sequences, maxlen=max_sequence_length, padding="post"
    )


text_train = prep_text(df_train["sentence"], tokenizer, max_sequence_length)
text_test = prep_text(df_test["sentence"], tokenizer, max_sequence_length)
text_val = prep_text(df_val["sentence"], tokenizer, max_sequence_length)

tag_train = prep_text(df_train["tag"], tag_tokenizer, max_sequence_length)
tag_test = prep_text(df_test["tag"], tag_tokenizer, max_sequence_length)
tag_val = prep_text(df_val["tag"], tag_tokenizer, max_sequence_length)

In [14]:
tag_train.shape
tag_train[0]

array([ 2,  2,  7, 11,  5,  6,  7, 20, 12,  4,  1,  3,  4,  6,  1,  2, 11,
        8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [15]:
### Encode the tags in 1hot encoding

num_classes = len(tag_tokenizer.word_index) + 1
tag_categorical_train = to_categorical(tag_train, num_classes)
tag_categorical_test = to_categorical(tag_test, num_classes)
tag_categorical_val = to_categorical(tag_val, num_classes)

In [ ]:
all_classes = list(tag_tokenizer.word_index.keys())
all_tokens = list(tag_tokenizer.word_index.values())
punct_classes = [",", ".", ":", "``", "''", "$", "#", "sym", "-rrb-", "-lrb-"]
punct_tokens = [tag_tokenizer.word_index[p] for p in punct_classes]
allowed_classes = [
    word for word in tag_tokenizer.index_word.values() if word not in punct_classes
]
allowed_tokens = [token for token in all_tokens if token not in punct_tokens]

print(f"Tags: {all_classes}")
print(f"All tag-tokens: {all_tokens}\n")
print(f"Punctuations: {punct_classes}")
print(f"Tokenized punctuations {punct_tokens}\n")
print(f"Tags without punctuation: {allowed_classes}")
print(f"Tokens will be used in evaluations: {allowed_tokens}")

In [ ]:
### Create the embedding matrix
embeddings_index = {}
with open(glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
num_words_in_embedding = 0
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        num_words_in_embedding += 1
        embedding_matrix[i] = embedding_vector

In [ ]:
### Inspect tokens' embedding vectors
idx_token = 2
print(
    f"Token: {list(tokenizer.word_index.keys())[idx_token]} \nVector: {embedding_matrix[idx_token]}"
)

In [ ]:
# TODO fare un po' di inspection.

# [Task 3 - 1.0 points] Model definition

You are now tasked to define your neural POS tagger.

### Instructions

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.
* You are **free** to experiment with hyper-parameters to define the baseline model.

* **Model 1**: add an additional LSTM layer to the Baseline model.
* **Model 2**: add an additional Dense layer to the Baseline model.

* **Do not mix Model 1 and Model 2**. Each model has its own instructions.

**Note**: if a document contains many tokens, you are **free** to split them into chunks or sentences to define your mini-batches.

In [ ]:
def get_model(lstm_units=64, add_lstm=False, add_dense=False):
    number_of_words = len(tokenizer.word_index.keys())
    number_of_tags = len(tag_tokenizer.word_index.keys())

    input = Input(shape=(max_sequence_length,))

    embedding_layer = Embedding(
        input_dim=number_of_words + 1,
        output_dim=embedding_dim,
        input_length=max_sequence_length,
        weights=[embedding_matrix],
        trainable=False,
    )(input)
    bi_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(embedding_layer)
    bi_lstm2 = (
        Bidirectional(LSTM(lstm_units, return_sequences=True))(bi_lstm)
        if add_lstm
        else bi_lstm
    )
    dense2 = (
        TimeDistributed(Dense(number_of_tags + 1, activation="relu"))(bi_lstm2)
        if add_dense
        else bi_lstm
    )
    dense_output = TimeDistributed(Dense(number_of_tags + 1, activation="softmax"))(
        dense2
    )
    model = Model(input, dense_output)

    ### No logits: we have 1 hot encoding as labels
    model.compile(
        loss=CategoricalCrossentropy(from_logits=False),
        optimizer=Adam(5e-3),
        metrics=["accuracy"],
    )
    # model.summary()
    return model

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    mode="max",
    patience=20,
    restore_best_weights=True,
    min_delta=0.001,
)
reduce_lr = ReduceLROnPlateau(
    monitor="val_accuracy",
    mode="max",
    factor=0.1,
    patience=7,
    cooldown=1,
    min_lr=1e-5,
    min_delta=0.001,
)

# [Task 4 - 1.0 points] Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using macro F1-score, compute over **all** tokens.
* **Concatenate** all tokens in a data split to compute the F1-score. (**Hint**: accumulate FP, TP, FN, TN iteratively)
* **Do not consider punctuation and symbol classes** $\rightarrow$ [What is punctuation?](https://en.wikipedia.org/wiki/English_punctuation)

**Note**: What about OOV tokens?
   * All the tokens in the **training** set that are not in GloVe are **not** considered as OOV
   * For the remaining tokens (i.e., OOV in the validation and test sets), you have to assign them a **static** embedding.
   * You are **free** to define the static embedding using any strategy (e.g., random, neighbourhood, etc...)

In [ ]:
def metric(model, text, tag_categorical, labels=None):
    # TODO what is a static embedding?

    # TODO controllare che questo vada bene: fare la prova accumulando manualmente TP FP TN FN

    y_pred = model.predict([text]).argmax(-1).flatten()
    y_test_flatten = tag_categorical.argmax(-1).flatten()
    score = f1_score(
        y_test_flatten,
        y_pred,
        labels=labels,  # allowed tokens
        average="macro",
        zero_division=0,
    )

    return score

# [Task 5 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate the Baseline, Model 1, and Model 2.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.

In [ ]:
def experiment(seed, model1=False, model2=False):
    set_reproducibility(seed)

    model = get_model(add_lstm=model1, add_dense=model2)

    ### train model on the train set
    history = model.fit(
        text_train,
        tag_categorical_train,
        batch_size=128,
        epochs=50,
        validation_data=(text_val, tag_categorical_val),
        callbacks=[early_stopping, reduce_lr],
    )

    ### compute metrics
    score = metric(model, text_val, tag_categorical_val)

    return score, history, model

In [ ]:
seeds = [333, 666, 999]
seeds = [333]

In [ ]:
### baseline
baseline_results = [experiment(s) for s in seeds]
baseline_scores = [r[0] for r in baseline_results]
baseline_histories = [r[1] for r in baseline_results]
baseline_models = [r[2] for r in baseline_results]

In [ ]:
[print(h) for h in baseline_scores]
[plt.plot(h.history["loss"]) for h in baseline_histories]

plt.show()

In [ ]:
### model1
model1_results = [experiment(s, model1=True) for s in seeds]
model1_scores = [r[0] for r in model1_results]
model1_histories = [r[1] for r in model1_results]
model1_models = [r[2] for r in model1_results]

In [ ]:
[print(h) for h in model1_scores]
[plt.plot(h.history["loss"]) for h in model1_histories]
plt.show()

In [ ]:
### model2
model2_results = [experiment(s, model2=True) for s in seeds]
model2_scores = [r[0] for r in model2_results]
model2_histories = [r[1] for r in model2_results]
model2_models = [r[2] for r in model2_results]

In [ ]:
[print(h) for h in model2_scores]
[plt.plot(h.history["loss"]) for h in model2_histories]
plt.show()

In [ ]:
### compute metrics on validation set

baseline_val_metrics = [
    metric(m, text_val, tag_categorical_val, labels=allowed_tokens)
    for m in baseline_models
]
model1_val_metrics = [
    metric(m, text_val, tag_categorical_val, labels=allowed_tokens)
    for m in model1_models
]
model2_val_metrics = [
    metric(m, text_val, tag_categorical_val, labels=allowed_tokens)
    for m in model2_models
]

# facendo la media: non credo abbia senso: possiamo fare la media degli score ma non la media dei modelli (dei parametri)
# e adesso dovrei prendere 3 modelli e fare la media (come?)


# senza fare la media
# ogni modello ha 3 chances di fare bene, poi prendiamo il cadidato migliore

# se dopo lo dovessimo allenare di nuovo (meglio, di più, su più dati) allora
# avrebbe senso prendere L'ARCHITETTURA che ha dato la media migliore
# una specie di genetic programming

# ci deve essere un modo più efficiente (meno memoria) per fare questa cosa
# non voglio concatenare le liste
### pick best performing model
names = [
    f"{model}_{seed}" for model in ["baseline", "model1", "model2"] for seed in seeds
]
d = {}
# metti in parallelo le liste
for model_list, score_list, name_list in zip(
    [baseline_models, model1_models, model2_models],
    [baseline_val_metrics, model1_val_metrics, model2_val_metrics],
    [
        [f"baseline_{s}" for s in seeds],
        [f"model1_{s}" for s in seeds],
        [f"model2_{s}" for s in seeds],
    ],
):
    # itera sugli elementi di ogni lista
    for m, s, n in zip(model_list, score_list, name_list):
        d.update({s: (m, n)})

val_score = max(d)
best_model, best_model_name = d[val_score]

print(f"best model: {best_model_name}")

# report: commento sui risultati: il training è un po' rumoroso (si può migliorare),

# [Task 6 - 1.0 points] Error Analysis

You are tasked to evaluate your best performing model.

### Instructions

* Compare the errors made on the validation and test sets.
* Aggregate model errors into categories (if possible)
* Comment the about errors and propose possible solutions on how to address them.

In [ ]:
### compare errors made on validation and test set
test_score = metric(best_model, text_test, tag_categorical_test, labels=allowed_tokens)
print(test_score)

val_score_per_cat = [
    metric(best_model, text_val, tag_categorical_val, labels=[t])
    for t in allowed_tokens
]
test_score_per_cat = [
    metric(best_model, text_test, tag_categorical_test, labels=[t])
    for t in allowed_tokens
]

### aggregate model erors into categories

# spero che categories siano quelle della classificazione e non "categorie di errori" tipo FP FN
print("TAG\tval\ttest")
for c, v, t in zip(allowed_classes, val_score_per_cat, test_score_per_cat):
    print(f"{c}\t{v}\t{t}")

# propose solutions:  qualche categoria manca qui e là

In [ ]:
### Predict on validation and test set
### argmax transforms from categorical to numbers

tag_pred_val = best_model.predict([text_val]).argmax(-1)
tag_pred_test = best_model.predict([text_test]).argmax(-1)

In [ ]:
def get_data_distribution(data):
    unique, counts = np.unique(data, return_counts=True)
    counts = dict(zip(unique, counts))
    # TODO: li sorto sulla label per far vedere le differenze di frequenza
    counts = dict(sorted(counts.items(), key=lambda item: item[0], reverse=False))

    ### If the tag_token is not allowed then delete it.
    for key in unique:
        if key not in allowed_tokens:
            del counts[key]
    labels = [tag_tokenizer.index_word[index] for index in counts.keys()]
    result = dict(zip(labels, counts.values()))
    return result


def plot_data_distribution(data, name):
    counts = get_data_distribution(data)
    keys = list(counts.keys())
    values = list(counts.values())
    fig = plt.figure(figsize=(6, 6))
    plt.barh(keys, values, color="#902e59")
    plt.title(f"Class Distribution of {name}")
    plt.ylabel("Tags")
    plt.xlabel("Count")
    plt.tick_params(labelsize=8)
    plt.show()

In [ ]:
plot_data_distribution(tag_train, name="Train Set")

In [ ]:
plot_data_distribution(tag_val, name="Validation Set")

In [ ]:
plot_data_distribution(tag_test, name="Test Set")

In [ ]:
def get_cf_report(tag_pred, tag):
    cf_report_val = pd.DataFrame(
        classification_report(
            tag_pred.flatten(),
            tag.flatten(),
            labels=all_tokens,
            target_names=all_classes,
            zero_division=0,
            output_dict=True,
        )
    ).transpose()
    cf_report_val.reset_index(inplace=True)
    cf_report_val.columns = ["tags", "precision", "recall", "f1-score", "support"]

    print(
        "F1 score calculated on validation set:",
        f1_score(
            tag_pred.flatten(),
            tag.flatten(),
            labels=all_tokens,
            zero_division=0,
            average="macro",
        ),
        "\n",
    )
    return cf_report_val

In [ ]:
### Display classification report for Validation Set
cf_report_val = get_cf_report(tag_pred_val, tag_val)
display(cf_report_val)

In [ ]:
### Display classification report for Test Set
cf_report_test = get_cf_report(tag_pred_test, tag_test)
display(cf_report_test)

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Trainable Embeddings

You are **free** to define a trainable or non-trainable Embedding layer to load the GloVe embeddings.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).

However, you are **free** to play with their hyper-parameters.

### Neural Libraries

You are **free** to use any library of your choice to implement the networks (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Keras TimeDistributed Dense layer

If you are using Keras, we recommend wrapping the final Dense layer with `TimeDistributed`.

### Error Analysis

Some topics for discussion include:
   * Model performance on most/less frequent classes.
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

### Punctuation

**Do not** remove punctuation from documents since it may be helpful to the model.

You should **ignore** it during metrics computation.

If you are curious, you can run additional experiments to verify the impact of removing punctuation.

# The End